In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd

In [ ]:
def remove_special_characters(text):
    text = text.replace('\n', ' ')
    text = ' '.join(text.split())
    return text

In [ ]:
def get_job_postings(keyword, location, days):
  stop = False
  hrs = days * 86400
  start = 0
  keyword = keyword.replace(' ', '%20')
  location = location.replace(' ', '%20')
  processed_posts = []
  error_pages = []
  while stop!=True:
    pre_url = 'https://linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?f_TPR=r'+str(hrs)+'&keywords='+keyword+'&location='+location
    final_url= pre_url + '&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0&start='+str(start)
    r = requests.get(final_url)
    if r.status_code == 200:
      soup = BeautifulSoup(r.content, 'html.parser')
      posts = soup.find_all('li')
      for post in posts:
        try:
          title = remove_special_characters(post.div.find(class_='base-search-card__info').h3.string) if post.div.find(class_='base-search-card__info') != None else ''
          company = remove_special_characters(post.div.find(class_='base-search-card__info').h4.a.string) if post.div.find(class_='base-search-card__info') != None else ''
          url = post.div.a['href'] if post.div != None else ''
          fetched_location = remove_special_characters(post.div.find(class_='job-search-card__location').string) if post.div.find(class_='job-search-card__location') != None else ''
          company_url = post.div.find(class_='base-search-card__info').h4.a['href'] if post.div.find(class_='base-search-card__info') != None else ''
          p = {
              'Title':title,
              'Company': company,
              'url':url,
              'Location':fetched_location,
              'company_url':company_url
          }
          processed_posts.append(p)
        except:
          error_pages.append(post)
    else:
      print(r.status_code)
      print('Stopped at page',str(start))
      print(final_url)
      stop = True
    start = start+25
  return [pd.DataFrame(processed_posts), error_pages]

In [ ]:
ml_search_keys = ['Data Science','Machine Learning','Artificial Intelligence']
country_list = ['Canada', 'India', 'United States']
postings = pd.DataFrame(columns=['Title', 'Company', 'url', 'Location', 'company_url'])
for search_key in ml_search_keys:
  for country in country_list:
    job_postings, error_posts = get_job_postings(search_key, country, 30)
    postings = pd.concat([postings, job_postings], ignore_index=True)

400
Stopped at page 1000
https://linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?f_TPR=r2592000&keywords=Data%20Science&location=Canada&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0&start=1000
400
Stopped at page 1000
https://linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?f_TPR=r2592000&keywords=Data%20Science&location=India&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0&start=1000
400
Stopped at page 1000
https://linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?f_TPR=r2592000&keywords=Data%20Science&location=United%20States&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0&start=1000
400
Stopped at page 1000
https://linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?f_TPR=r2592000&keywords=Machine%20Learning&location=Canada&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0&start=1000
400
Stopped at page 1000
https://linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/sea

In [ ]:
postings.to_csv('ml_postings.csv')

Function to Fetch Job Posting

In [ ]:
def get_job_description(url):
  rms = remove_special_characters
  r = requests.get(url)
  job_desciption = {}
  if r.status_code == 200:
    soup = BeautifulSoup(r.content, 'html.parser')
    try:
      if soup.find('span', class_='posted-time-ago__text') != None:
        job_desciption['posted'] = rms(soup.find('span', class_='posted-time-ago__text').string)
      if soup.find('figcaption', class_='num-applicants__caption') != None:
        job_desciption['applicants'] = rms(soup.find('figcaption', class_='num-applicants__caption').string)
      if soup.find('span', class_='num-applicants__caption') != None:
        job_desciption['applicants'] = rms(soup.find('span', class_='num-applicants__caption').string)
      if soup.find('ul',class_='description__job-criteria-list') != None:
        job_criteria_section = soup.find('ul',class_='description__job-criteria-list')
        job_criteria_list = job_criteria_section.find_all('li')
        extracted_criteria = {}
        for job_criteria in job_criteria_list:
          extracted_criteria[rms(job_criteria.h3.string)] = rms(job_criteria.span.string)
        job_desciption['criteria'] = extracted_criteria
      if soup.find('section',class_="core-section-container") != None:
        description = soup.find('section',class_="core-section-container").strings
        sentences = []
        for s in description:
          sentences.append(rms(s))
        job_desciption['text_desciption'] = sentences
    except Exception:
       print(Exception)
       return 'Exception'
  else:
    print('Not 200 for', url)
  return job_desciption

Extracting list from saved file

In [ ]:
postings

,Title,Company,url,Location,company_url
0,Data Analyst,Peggy,https://ca.linkedin.com/jobs/view/data-analyst...,"Toronto, Ontario, Canada",https://www.linkedin.com/company/peggy-com?trk...
1,Data Scientist,Ample Insight,https://ca.linkedin.com/jobs/view/data-scienti...,"Toronto, Ontario, Canada",https://ca.linkedin.com/company/ample-insight?...
2,Data Scientist (Optimization),Tiger Analytics,https://ca.linkedin.com/jobs/view/data-scienti...,"Toronto, Ontario, Canada",https://www.linkedin.com/company/tiger-analyti...
3,Data Scientist,Scene+,https://ca.linkedin.com/jobs/view/data-scienti...,"Greater Toronto Area, Canada",https://ca.linkedin.com/company/sceneplusrewar...
4,Data Scientist - Toronto,Canacre,https://ca.linkedin.com/jobs/view/data-scienti...,"Toronto, Ontario, Canada",https://ca.linkedin.com/company/canacre?trk=pu...
...,...,...,...,...,...
8268,Copy Writer - Freelance [Remote],Braintrust,https://www.linkedin.com/jobs/view/copy-writer...,"Boston, MA",https://www.linkedin.com/company/usebraintrust...
8269,Vice President of Engineering,ABATECH,https://www.linkedin.com/jobs/view/vice-presid...,"Fort Lauderdale, FL",https://co.linkedin.com/company/aba-tech?trk=p...
8270,Power BI Developer,Adame Services,https://www.linkedin.com/jobs/view/power-bi-de...,"Sacramento, CA",https://www.linkedin.com/company/adameservices...
8271,AI/Data/Visualization Department Manager,ecocareers,https://www.linkedin.com/jobs/view/ai-data-vis...,"Idaho Falls, ID",https://www.linkedin.com/company/ecocareers?tr...


In [ ]:
postings = pd.read_csv('ml_postings.csv')

In [ ]:
postings['url'] = postings.apply(lambda x: x.url.split('?')[0], axis=1)

In [ ]:
postings['id'] = postings.apply(lambda x: x.url.split('-')[len(x.url.split('-'))-1], axis=1)

In [ ]:
postings.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
postings = postings.drop_duplicates(subset=['id'])

In [ ]:
postings

,Title,Company,url,Location,company_url,id
0,Data Analyst,Peggy,https://ca.linkedin.com/jobs/view/data-analyst...,"Toronto, Ontario, Canada",https://www.linkedin.com/company/peggy-com?trk...,3668730857
1,Data Scientist,Ample Insight,https://ca.linkedin.com/jobs/view/data-scienti...,"Toronto, Ontario, Canada",https://ca.linkedin.com/company/ample-insight?...,3668302825
2,Data Scientist (Optimization),Tiger Analytics,https://ca.linkedin.com/jobs/view/data-scienti...,"Toronto, Ontario, Canada",https://www.linkedin.com/company/tiger-analyti...,3668126664
3,Data Scientist,Scene+,https://ca.linkedin.com/jobs/view/data-scienti...,"Greater Toronto Area, Canada",https://ca.linkedin.com/company/sceneplusrewar...,3670068034
4,Data Scientist - Toronto,Canacre,https://ca.linkedin.com/jobs/view/data-scienti...,"Toronto, Ontario, Canada",https://ca.linkedin.com/company/canacre?trk=pu...,3670303702
...,...,...,...,...,...,...
8263,Part Time - Clinical Robot Associate - TH-HEB ...,Diligent Robotics,https://www.linkedin.com/jobs/view/part-time-c...,"Bedford, TX",https://www.linkedin.com/company/diligent-robo...,3668302176
8265,Research Associate-Fixed Term,Michigan State University,https://www.linkedin.com/jobs/view/research-as...,"East Lansing, MI",https://www.linkedin.com/company/michigan-stat...,3674124701
8266,Back-End Developer (Entry Level) - US/Canada,Patterned Learning AI,https://www.linkedin.com/jobs/view/back-end-de...,"New York, NY",https://www.linkedin.com/company/patterned-lea...,3672387114
8271,AI/Data/Visualization Department Manager,ecocareers,https://www.linkedin.com/jobs/view/ai-data-vis...,"Idaho Falls, ID",https://www.linkedin.com/company/ecocareers?tr...,3670361250


Exctracting Description

In [ ]:
criteria_list = ['Employment type','Seniority level', 'Job function', 'Industries']
postings_with_description = postings.copy()
postings_with_description['description'] = None
for index, row in postings.iterrows():
  url = row['url']
  description = get_job_description(url)
  if len(description.keys())!=0:
    if 'posted' in description:
      postings_with_description.loc[index, 'posted'] = description['posted']
    if 'applicants' in description:
      postings_with_description.loc[index, 'applicants'] = description['applicants']
    if 'criteria' in description:
      for criteria in criteria_list:
        if criteria in description['criteria']:
          postings_with_description.loc[index, criteria]  = description['criteria'][criteria]
    if 'text_desciption' in description:
      list_of_strings = [x for x in description['text_desciption'] if len(x)!= 0]
      description_string = ' '.join(list_of_strings)
      postings_with_description.loc[index, 'Description String']  = description_string
  if index%5 == 0:
    time.sleep(5)

In [ ]:
postings_with_description.to_pickle('ml_job_descriptions.pkl')

In [ ]:
postings_with_description['id'] = postings_with_description['id'].astype(int)

In [ ]:
import re
def change_domain(url):
  pattern = r"https://([a-z]+)\.linkedin\.com"
  match = re.search(pattern, url)
  if match:
    converted_url = re.sub(pattern, f"https://www.linkedin.com", url)
    return converted_url

Finding Missing

In [ ]:
count_missing = 0
for index, row in postings_with_description.iterrows():
  url = row['url']
  posted = row['posted']
  if type(posted) != str:
    count_missing = count_missing+1
    converted_url = change_domain(url)
    description = get_job_description(converted_url)
    if len(description.keys())!=0:
      if 'posted' in description:
        postings_with_description.loc[index, 'posted'] = description['posted']
      if 'applicants' in description:
        postings_with_description.loc[index, 'applicants'] = description['applicants']
      if 'criteria' in description:
        for criteria in criteria_list:
          if criteria in description['criteria']:
            postings_with_description.loc[index, criteria]  = description['criteria'][criteria]
      if 'text_desciption' in description:
        list_of_strings = [x for x in description['text_desciption'] if len(x)!= 0]
        description_string = ' '.join(list_of_strings)
        postings_with_description.loc[index, 'Description String']  = description_string
      if count_missing%5 ==0:
        time.sleep(5)
count_missing

Not 200 for https://www.linkedin.com/jobs/view/react-developer-entry-level-us-at-patterned-learning-ai-3674314717


1

Counting Missing

In [ ]:
postings_with_description.to_pickle('ml_job_descriptions.pkl')

In [ ]:
postings_with_description

,Title,Company,url,Location,company_url,id,description,posted,applicants,Employment type,Seniority level,Job function,Industries,Description String
0,Data Analyst,Peggy,https://ca.linkedin.com/jobs/view/data-analyst...,"Toronto, Ontario, Canada",https://www.linkedin.com/company/peggy-com?trk...,3668730857,None,4 days ago,Over 200 applicants,Full-time,Entry level,Information Technology,Artists and Writers,Peggy is reenvisioning the art market by estab...
1,Data Scientist,Ample Insight,https://ca.linkedin.com/jobs/view/data-scienti...,"Toronto, Ontario, Canada",https://ca.linkedin.com/company/ample-insight?...,3668302825,None,4 days ago,Over 200 applicants,Full-time,Associate,Engineering and Information Technology,Data Infrastructure and Analytics,Company Description You will join a world-clas...
2,Data Scientist (Optimization),Tiger Analytics,https://ca.linkedin.com/jobs/view/data-scienti...,"Toronto, Ontario, Canada",https://www.linkedin.com/company/tiger-analyti...,3668126664,None,6 days ago,Over 200 applicants,Full-time,Mid-Senior level,Consulting,"Technology, Information and Internet",Tiger Analytics is looking for experienced Dat...
3,Data Scientist,Scene+,https://ca.linkedin.com/jobs/view/data-scienti...,"Greater Toronto Area, Canada",https://ca.linkedin.com/company/sceneplusrewar...,3670068034,None,2 days ago,87 applicants,Full-time,Entry level,Engineering and Information Technology,Marketing Services,Who We Are Join Scene+ and be part of Canada's...
4,Data Scientist - Toronto,Canacre,https://ca.linkedin.com/jobs/view/data-scienti...,"Toronto, Ontario, Canada",https://ca.linkedin.com/company/canacre?trk=pu...,3670303702,None,1 week ago,56 applicants,Full-time,Entry level,Engineering and Information Technology,Utilities,Join the Canacre team in an exciting role work...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8263,Part Time - Clinical Robot Associate - TH-HEB ...,Diligent Robotics,https://www.linkedin.com/jobs/view/part-time-c...,"Bedford, TX",https://www.linkedin.com/company/diligent-robo...,3668302176,None,4 days ago,Be among the first 25 applicants,Part-time,Entry level,"Research, Analyst, and Information Technology","Technology, Information and Internet",Clinical Robot Associate - Part Time (3 8-Hour...
8265,Research Associate-Fixed Term,Michigan State University,https://www.linkedin.com/jobs/view/research-as...,"East Lansing, MI",https://www.linkedin.com/company/michigan-stat...,3674124701,None,3 weeks ago,Be among the first 25 applicants,Part-time,Mid-Senior level,"Research, Analyst, and Information Technology",Higher Education,Job postings Research Associate-Fixed Term Bac...
8266,Back-End Developer (Entry Level) - US/Canada,Patterned Learning AI,https://www.linkedin.com/jobs/view/back-end-de...,"New York, NY",https://www.linkedin.com/company/patterned-lea...,3672387114,None,1 day ago,64 applicants,Full-time,Entry level,Engineering and Information Technology,IT Services and IT Consulting,"Back-End Developer (Entry Level) - US/Canada, ..."
8271,AI/Data/Visualization Department Manager,ecocareers,https://www.linkedin.com/jobs/view/ai-data-vis...,"Idaho Falls, ID",https://www.linkedin.com/company/ecocareers?tr...,3670361250,None,4 days ago,Be among the first 25 applicants,Full-time,Entry level,Information Technology,Staffing and Recruiting,Estimated pay range Salary information is not ...
